In [ ]:
import numpy as np
import numpy.random as rd
import torch
from torch import nn
from torch import optim
import torch.utils.data as load_data
import torchvision
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load MNIST data
train_data = torchvision.datasets.MNIST('MNIST/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))
test_data = torchvision.datasets.MNIST('MNIST/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

# Deep Learning Tutorial Part II: Deep Feed-Forward Networks

Let's now move on to classifying the full set of digits. This is now a much harder tasks, not only because we're now doing 10-way classification (instead of just binary classification as above), but also because many of the other hand-written digits look much more similar to each other than 1 and 0. For example, hand-written 5's often look quite similar to 6's, and 2's look like 7's.

To deal with this added layer of complexity, we'll thus turn to a deep network, which is a much more powerful model than the simple linear-sigmoidal model used above:
\begin{gather*}
    \mathbf{h}^{(1)} = \phi\left( \mathbf{W}^{(1)} \mathbf{x} \right) \\
    \mathbf{h}^{(2)} = \phi\left( \mathbf{W}^{(2)} \mathbf{h}^{(1)} \right) \\
    \vdots \\
    \mathbf{h}^{(L)} = \phi\left( \mathbf{W}^{(L)} \mathbf{h}^{(L-1)} \right) \\
    \mathbf{y} = \phi\left( \mathbf{W}^{out} \mathbf{h}^{(L)} \right) \\
    p_i = g_i(\mathbf{y})
\end{gather*}
where 
\begin{equation}
    g_i(\mathbf{y}) = \frac{e^{-y_i}}{\sum_{j=1}^{10} e^{-y_j}}
\end{equation}
is called the **softmax function** and serves the same purpose as the sigmoid funtion in the logistic regression example in part 1, but now for multi-class classification rather than just binary. The **activation function** $\phi(\cdot)$ is just some non-linear function. In the below we'll use linear rectification, commonly called **ReLU**, for rectified linear unit, in the machine learning community:
\begin{equation}
    \phi(x) = 
    \begin{cases}
        x & \text{if } x > 0 \\
        0 & \text{else}
    \end{cases}
\end{equation}

This model is depicted in the figure below. The parameters of this model are again the weights $\mathbf{W}^{(i)}$ between successive layers, and we'll learn them by using stochastic gradient descent on the negative log probability of the training data classes. 
<img src="deep_net.png" style="height: 300px;"/>
We refer to the $h_i^j$ units in the middle layers as **hidden units**, since they are in a sense "hidden" from the inputs and outputs of the network.

You can easily see why such an architecture is called a **deep** neural network. It is **feed-forward** because the flow of information from input to output just goes in one direction: each layer gets input from the previous layer only and passes its activity onto the next layer only. This contrasts with **recurrent networks** (part 3 of this tutorial), in which each "layer" passes its activity onto itself as well as onto others.

Here is some PyTorch code for building a deep feed-forward network:

In [ ]:
# Build Deep Network with three fully connected hidden layers using ReLU non-linearity
class DeepNet(nn.Module):
    def __init__(self):
        super(DeepNet, self).__init__()
        self.layer1 = nn.Linear(784, 500) # layer 1
        self.layer2 = nn.Linear(500, 500) # layer 2
        self.layer3 = nn.Linear(500, 200) # layer 3
        self.out = nn.Linear(200, 10) # output layer

    def forward(self, x):
        x = x.reshape(x.shape[0], -1) # need to reshape image into a vector
        h = torch.relu(self.layer1(x)) # hidden layer 1
        h = torch.relu(self.layer2(h)) # hidden layer 2
        h = torch.relu(self.layer3(h)) # hidden layer 3
        y = torch.relu(self.out(h)) # output layer y
        logp = torch.log_softmax(y, 0) # log probabilities from softmax
        return logp # return log p to make it easier to evaluate loss function

## 2.1) Train and test the deep network
Complete the code below for training this deep network. As opposed to part 1, in this case we'll use the whole data set and classify all 10 digits. Because this model is much larger and has many more parameters that need to be learnt, we'll have to use many more iterations of stochastic gradient descent.

In particular, what we'll do is actually perform gradient descent on enough batches to go through the whole training data set several times. This is done below using built-in PyTorch functions that sample random batches from the entire data set. One whole pass through the data set is called an **epoch**. You'll have to train for at least 3-4 epochs below for the loss function to really hit its floor. 

After training, plot the loss as a function of iterations. Also, test the model on some new unseen test data. What percentage of these test images are classified correctly? Some code is provided for this.

Try playing around with the SGD parameters and the model structure. For example, try changing the sizes of the hidden layers, changing the non-linearity from ReLU to something else (see [here](https://pytorch.org/docs/stable/nn.html#non-linear-activation-functions) for different PyTorch built-in non-linear activation functions), or adding in other types of layers (e.g. convolutions, normalization, max pooling...see [here](https://pytorch.org/docs/stable/nn.html#) for a whole zoo of layer types). How do these changes affect the final test performance?

In [ ]:
# Initialize network
deep_model = DeepNet()

# Set up SGD parameters
n_epochs = 4
alpha = .01
optimizer = optim.SGD(deep_model.parameters(), lr=alpha) # this function will perform SGD for us!

# Set up loss function
loss_fn = nn.NLLLoss() # built-in negative log-likelihood loss function, takes log probability as input

# Function for sampling batches from training data
batch_size = 100
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [ ]:
# Placeholder for loss
track_loss = []

# Loop over all batches in data set
for i in range(n_epochs):
    print('running epoch %i/%i...' % (i+1, n_epochs))
    for batch_images, batch_labels in train_loader: # sample batch
        # Run the model on this batch
        p = deep_model(batch_images)
        # Compute the loss
        loss = loss_fn(p, batch_labels)
        track_loss.append(loss.item())
        # Compute the gradient
        optimizer.zero_grad()
        loss.backward()
        # Update weights
        optimizer.step()

In [ ]:
# Plot the loss
fig, ax = plt.subplots()
ax.plot(track_loss)
ax.set_xlabel('iterations of stochastic gradient descent')
ax.set_ylabel('loss')

In [ ]:
# Function for sampling batches from test data
batch_size_test = 1000
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size_test, shuffle=True)

# Placeholder
test_acc = []

# Loop over all batches in test data
for image, label in test_loader: # sample batch
    # Run the model on this batch
    p = deep_model(image)
    # Classify
    c = np.argmax(p.detach().numpy(), axis=1)
    # Check if correct
    test_acc.append(c == label.detach().numpy())

# Calculate number of correctly classified images
test_acc = np.array(test_acc).flatten()
n_correct = test_acc.sum()
n_total = len(test_acc)
print('Correctly classified %i/%i images (%.2f%% accuracy)' % (n_correct, n_total, 100 * n_correct / n_total))

## 2.2) Visualize internal representations

How is the network doing classification? One approach to trying to understand the algorithm implemented by the trained model is to look at its internal representations. We can think of this deep network as applying a sequence of transformations to the inputted image. At each layer, the set of neuron activities is a kind of neural representation of the transformed image at that stage. The sequence of these representations should be such that it becomes easier and easier to tell the different hand-written digits apart, so that in the last layer it is easy to perform the classification.

To see how these representations are changing across the different layers of the network, below we'll actually visualize these representations by plotting them in two dimensions. The way we'll do this is by using PCA to reduce the dimensionality of these representations to two. We'll plot the dimensionality-reduced neural activations in each layer produced in response to images of a few digits. This will allow us to get some intuition for how the represntations of these digits change across layers.

We provide all the necessary code below. Try playing with the network as you did above and see how this changes the neural representations. One simple thing to try is to re-initialize the network and compare its representations before and after learning. Does any particularly relevant structure arise in the representations after learning?

In [ ]:
from sklearn.decomposition import PCA # function for doing PCA in one line of code!

In [ ]:
# Get layer names and create a placeholder dict
layer_names = [p[0] for p in deep_model._modules.items()]
layer_activations = {}
for layer in layer_names:
    layer_activations[layer] = []

# Write function to tell PyTorch to save activity in each layer to layer_activations dict
def get_activation(name):
    def hook(model, input, output):
        for tensor_i in output.detach():
            layer_activations[name].append(np.array(tensor_i.view(-1)))
    return hook

# Use 'hooks' to force PyTorch to run this function on each run of the model
for layer, name in zip(deep_model.children(), layer_names):
    layer.register_forward_hook(get_activation(name))

In [ ]:
# Run network on some images
data_loader = torch.utils.data.DataLoader(test_data, batch_size=1000, shuffle=True)
image_labels = []
for image, label in data_loader:
    # Run model to save activations
    outputs = deep_model(image)
    # Save labels
    [image_labels.append(lab) for lab in label]
image_labels = np.array(image_labels)

In [ ]:
fig, axs = plt.subplots(1, len(layer_names), figsize=(5 * len(layer_names), 5), sharex=True, sharey=True)

# Set up PCA
pca = PCA(n_components=2) # reduce dimensionality to 2

# Choose digits whose representations to plot and colors to plot them with
plot_digits = [5, 6, 9]
plot_colors = ['r', 'b', 'g']

# For each layer, reduce dimensionality of activations using PCA 
# and plot the dimensionality-reduced activations for a couple of digits
for i, (layer_name, layer_activity) in enumerate(layer_activations.items()):
    axs[i].set_title(layer_name)
    pc_activity = pca.fit_transform(layer_activity) # PCA dimensionality reduction in one line of code!
    # Plot representations of plot_digits
    for digit, color in zip(plot_digits, plot_colors):
        indx = (image_labels == digit)
        x, y = pc_activity[indx, 0], pc_activity[indx, 1]
        axs[i].plot(x, y, 'o', color=color, alpha=0.1)
        axs[i].plot(np.mean(x), np.mean(y), 'o', color=color, ms=10, markeredgecolor='k', label=digit)
    axs[i].set_xticks([])
    axs[i].set_yticks([])
axs[-1].legend()